In [5]:
import pandas as pd
from xp import *
from data_utils import load_csv

In [6]:
import pandas as pd
from xp import *
from data_utils import load_csv

root = '../data/Fantasy-Premier-League/data/2024-25'

fixtures_df = load_csv(f'{root}/fixtures.csv')
teams_df = load_csv(f'{root}/teams.csv')
players_df = load_csv(f'{root}/players_raw.csv')
xp_predictor = ScorePredictor('2024-25', players_df, teams_df, fixtures_df)
train_df = load_csv("../data/stats_by_game_played_with_home_away.csv")
xp_predictor = ScorePredictor('2024-25', players_df, teams_df, fixtures_df)

In [9]:
import numpy as np

# Keys that must match exactly or within float tolerance
FEATURE_KEYS = [
    'n_games_in_gw', 'points_scored', 'avg_home',
    'playing_against_mean_difficulty', 'playing_against_mean_defence',
    'playing_against_mean_attack', 'playing_for_difficulty',
    'playing_for_defence', 'playing_for_attack', 'form', 'position',
    'creativity', 'influence', 'threat', 'selected', 'playing_chance',
    'corners_and_free_kicks_order', 'penalties_order',
    'avg_minutes_when_playing', 'avg_points_when_playing',
    'avg_yellows_p_game_when_playing', 'avg_reds_p_game_when_playing',
    'avg_bps_p_game_when_playing', 'avg_goals_p_game_when_playing',
    'avg_assists_p_game_when_playing', 'avg_clean_sheets_when_playing',
    'avg_goals_conceded_when_playing', 'avg_starts_when_playing',
    'avg_saves_when_playing'
]

# Filter to 2024-25 season and sample a few rows
df_2425 = train_df[train_df["year"] == "2024-25"].copy()
samples = df_2425.sample(100, random_state=42)

for _, row in samples.iterrows():
    # Find player ID
    match = players_df[
        (players_df["first_name"] + " " + players_df["second_name"]).str.lower()
        == row["player"].lower()
    ]
    if match.empty:
        print(f"⚠️ Skipping {row['player']} GW{row['gw']} — no match in players_raw.csv")
        continue

    pid = int(match["id"].iloc[0])
    gw = int(row["gw"])

    # Run prepare_data
    try:
        new_data = xp_predictor.prepare_data(pid, gw)
    except Exception as e:
        print(f"❌ Error for {row['player']} GW{gw}: {e}")
        continue

    # Compare only the required keys
    mismatches = []
    for key in FEATURE_KEYS:
        old_val = row.get(key, None)
        new_val = new_data.get(key, None)
        if isinstance(old_val, (int, float, np.floating)):
            if not np.isclose(old_val, new_val, atol=1e-6, equal_nan=True):
                mismatches.append((key, old_val, new_val))
        else:
            if str(old_val) != str(new_val):
                mismatches.append((key, old_val, new_val))

    # Report
    if mismatches:
        print(f"\n🔍 Testing {row['player']} GW{gw} — {len(mismatches)} mismatches")
        for key, old, new in mismatches:
            print(f"  {key}: train_df={old} | prepare_data={new}")
    else:
        print(f"✅ {row['player']} GW{gw} — all {len(FEATURE_KEYS)} keys match")


✅ Matz Sels GW12 — all 29 keys match
✅ Pedro Porro GW11 — all 29 keys match
✅ Danny Welbeck GW29 — all 29 keys match
✅ Séamus Coleman GW18 — all 29 keys match
✅ Tyrick Mitchell GW5 — all 29 keys match
✅ Ola Aina GW13 — all 29 keys match
✅ Pedro Porro GW25 — all 29 keys match
✅ Joe Aribo GW12 — all 29 keys match
✅ Harry Winks GW28 — all 29 keys match
✅ Ryan Fraser GW8 — all 29 keys match
✅ Riccardo Calafiori GW13 — all 29 keys match
✅ Jack Taylor GW38 — all 29 keys match
✅ Harvey Barnes GW31 — all 29 keys match
✅ Jack Clarke GW26 — all 29 keys match
✅ Morgan Rogers GW32 — all 29 keys match
✅ André Onana GW4 — all 29 keys match
✅ Mats Wieffer GW13 — all 29 keys match
✅ Nélson Cabral Semedo GW8 — all 29 keys match
✅ Jack Stephens GW11 — all 29 keys match
✅ Benjamin White GW8 — all 29 keys match
✅ Carlos Soler GW16 — all 29 keys match
✅ Christopher Nkunku GW29 — all 29 keys match
✅ Adam Webster GW6 — all 29 keys match
✅ Mateus Gonçalo Espanha Fernandes GW35 — all 29 keys match
✅ Konstantin